<a href="https://colab.research.google.com/github/iakovkh/mypython/blob/main/LSTM_example33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Configuration section
data_file_name = '/content/sample_data/NN_cut2.xlsx'  # Path to the training data file
validate_file_name = '/content/sample_data/NN_control_02.xlsx'  # Path to the validation data file
normalize_data = True  # Flag to indicate whether to normalize the data
train_ratio = 0.8  # Ratio of data to be used for training
optimizer_name = 'Adam'  # Name of the optimizer to use (options: 'SGD', 'RMSprop', 'Adagrad', 'Adam')
loss_function_name = 'MSELoss'  # Name of the loss function to use (options: 'L1Loss', 'CrossEntropyLoss', 'MSELoss')
activation_function_name = 'Sigmoid'  # Name of the activation function to use (options: 'Sigmoid', 'Tanh', 'ReLU')
epochs = 15000  # Number of epochs for training
network_structure = [(1, 3), (2, 2), (3, 1), (4, 2)]  # Structure of the neural network (layer number, number of neurons)
display_network_structure = False  # Flag to indicate whether to display the network structure

# Step I - General data preparation
class MyMatrix:
    def __init__(self):
        self.data = None  # Initialize data attribute to None

    def read_data_from_file(self, file_name):
        self.data = pd.read_excel(file_name)  # Read data from an Excel file

    def normalize_data(self):
        scaler = MinMaxScaler()  # Create a MinMaxScaler object
        self.data = pd.DataFrame(scaler.fit_transform(self.data), columns=self.data.columns)  # Normalize the data

    def display_console(self, column=None):
        if column:
            print(self.data[column])  # Print a specific column of the data
        else:
            print(self.data)  # Print the entire data

    def display_visual(self, column=None):
        if column:
            self.data[column].plot()  # Plot a specific column of the data
            plt.show()  # Display the plot
        else:
            self.data.plot(subplots=True)  # Plot all columns of the data in separate subplots
            plt.show()  # Display the plots

# Step II – Read and prepare data
MyData = MyMatrix()  # Create an instance of MyMatrix
MyData.read_data_from_file(data_file_name)  # Read the training data from the file

if normalize_data:
    MyData.normalize_data()  # Normalize the data if the flag is set

train_size = int(len(MyData.data) * train_ratio)  # Calculate the size of the training data
train_data = MyData.data.iloc[:train_size]  # Split the data into training data
test_data = MyData.data.iloc[train_size:]  # Split the data into testing data

# Step III – Design the neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, network_structure, activation_function):
        super(NeuralNetwork, self).__init__()
        layers = []  # Initialize a list to hold the layers of the network
        for i, (layer_num, neurons) in enumerate(network_structure):
            if i == 0:
                layers.append(nn.Linear(input_size, neurons))  # Add the first layer with input size
            else:
                layers.append(nn.Linear(network_structure[i-1][1], neurons))  # Add subsequent layers
            if i < len(network_structure) - 1:
                layers.append(activation_function())  # Add activation function between layers
        self.network = nn.Sequential(*layers)  # Create a sequential container of layers

    def forward(self, x):
        return self.network(x)  # Define the forward pass

input_size = train_data.shape[1] - 2  # Determine the input size for the network
activation_function = getattr(nn, activation_function_name)  # Get the activation function
model = NeuralNetwork(input_size, network_structure, activation_function)  # Create the neural network model

if display_network_structure:
    print("Network Structure:")
    for i, (layer_num, neurons) in enumerate(network_structure):
        print(f"Layer {layer_num}: {neurons} neurons")  # Print the structure of the network
    print("Training Data:")
    print(train_data)  # Print the training data
    print("Testing Data:")
    print(test_data)  # Print the testing data

# Training the network
optimizer = getattr(optim, optimizer_name)(model.parameters())  # Get the optimizer
loss_function = getattr(nn, loss_function_name)()  # Get the loss function

train_inputs = torch.tensor(train_data.iloc[:, 2:].values, dtype=torch.float32)  # Prepare training inputs
train_outputs = torch.tensor(train_data.iloc[:, :2].values, dtype=torch.float32)  # Prepare training outputs
test_inputs = torch.tensor(test_data.iloc[:, 2:].values, dtype=torch.float32)  # Prepare testing inputs
test_outputs = torch.tensor(test_data.iloc[:, :2].values, dtype=torch.float32)  # Prepare testing outputs

losses = []  # Initialize a list to store the loss values
for epoch in range(epochs):
    optimizer.zero_grad()  # Zero the gradients
    outputs = model(train_inputs)  # Forward pass
    loss = loss_function(outputs, train_outputs)  # Calculate the loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights
    losses.append(loss.item())  # Store the loss value
    if epoch % 50 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item()}")  # Print the loss every 50 epochs

plt.plot(range(epochs), losses)  # Plot the loss values over epochs
plt.xlabel('Epochs')  # Label for x-axis
plt.ylabel('Loss')  # Label for y-axis
plt.title('Training Loss')  # Title of the plot
plt.show()  # Display the plot

# Step IV - Forecasting results
MyData.read_data_from_file(validate_file_name)  # Read the validation data from the file
if normalize_data:
    MyData.normalize_data()  # Normalize the validation data if the flag is set

validate_inputs = torch.tensor(MyData.data.iloc[:, 2:].values, dtype=torch.float32)  # Prepare validation inputs
predictions = model(validate_inputs).detach().numpy()  # Make predictions and convert to numpy array

output_df = pd.DataFrame(predictions, columns=['Predicted_Column1', 'Predicted_Column2'])  # Create a DataFrame for predictions
output_df.to_excel('output.xlsx', index=False)  # Save the predictions to an Excel file


Epoch 0/15000, Loss: 0.46077749133110046
Epoch 50/15000, Loss: 0.3847995698451996
Epoch 100/15000, Loss: 0.3283790946006775
Epoch 150/15000, Loss: 0.2852981388568878
Epoch 200/15000, Loss: 0.25215452909469604
Epoch 250/15000, Loss: 0.22767901420593262
Epoch 300/15000, Loss: 0.21066753566265106
Epoch 350/15000, Loss: 0.19963161647319794
Epoch 400/15000, Loss: 0.19297677278518677
Epoch 450/15000, Loss: 0.18925058841705322
Epoch 500/15000, Loss: 0.1873096376657486
Epoch 550/15000, Loss: 0.18636344373226166
Epoch 600/15000, Loss: 0.18592458963394165
Epoch 650/15000, Loss: 0.18572168052196503
Epoch 700/15000, Loss: 0.18561576306819916
Epoch 750/15000, Loss: 0.18553954362869263
Epoch 800/15000, Loss: 0.18545952439308167
Epoch 850/15000, Loss: 0.1853555291891098
Epoch 900/15000, Loss: 0.18521089851856232
Epoch 950/15000, Loss: 0.18500745296478271
Epoch 1000/15000, Loss: 0.184719055891037
Epoch 1050/15000, Loss: 0.18431386351585388
Epoch 1100/15000, Loss: 0.18379545211791992
Epoch 1150/15000, 